<a href="https://colab.research.google.com/github/ilyachti/Projectmatrix/blob/master/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0_%D0%B8_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D0%BE%D1%86%D0%B5%D0%BD%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорты библиотек

In [57]:
from scipy.stats import norm, t, sem
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple,List,Union
import pandas as pd
from random import choices
from tqdm import tqdm
import plotly.graph_objects as go
import random


# Задача №1 Точечное оценивание

В рамках задачи нужно посмотреть как меняется точность эмпирической точечной оценки на примере Орла и Решки для разного количества наблюдений

In [44]:
coin_list = ['Орел', 'Решка']
N = 5000 # Количество наблюдений

toDo <br>
Для проведения эксперимента использовать [choices](https://www.w3schools.com/python/ref_random_choices.asp)

In [45]:
def get_probs(coin_list:List[str],n:int) -> List[float]:

  probs = []
  count_heads = 0  # Счетчик 'Орлов'
  for i in range(1, n + 1):
        # Случайный выбор исхода (Орел или Решка)
        outcome = random.choices(coin_list, weights = [1, 1], k = i)
        count_heads = outcome.count("Орел")



        # Вычисляем текущую вероятность 'Орла'
        probs.append(count_heads*1.0 / i)


  return probs


probs = get_probs(
    coin_list = coin_list,
    n = N
)

In [46]:
# Запусти код для визуальной проверки сходимости
# Создаем график
fig = go.Figure()

# Линия вероятности по итогам эксперимента
fig.add_trace(go.Scatter(x=list(range(1, N)), y=probs, mode='lines', name='Probs'))

# Теоретическая вероятность наступления Орла
fig.add_trace(go.Scatter(x=[0, N], y=[0.5, 0.5], mode='lines', line=dict(color='red', dash='dash'), name='Теоретическая вероятность'))

fig.update_layout(
    title="Сходимость оценки вероятности Орла и Решки от количества наблюдений",
    xaxis_title="Количество испытаний",
    yaxis_title="Вероятность",
    showlegend=True
)

fig.show()

# Задача №2 Доверительное тестирование

In [58]:
# Создаем выборку роста
data = norm(165,10).rvs(10000)

In [59]:
def mean_confidence_interval(sample:Union[List,np.ndarray,pd.Series], confidence:float=0.95) -> Tuple[float]:
  """
  Функция для расчета доверительного интервала оценки среднего.

  :param sample: Массив наблюдений
  :type sample: List
  :param confidence: Уровень доверия (0 до 1)
  :type confidence: float
  :return: Возвращаем 3 значение: среднее, левая граница интервала, правая граница интервала
  :rtype: Tuple[float]

  """
  assert 0 < confidence < 1, "Недопустимое значение confidence. Значение должно быть (0,1)"
  # Расчет среднего
  mmean = np.mean(sample)
  # Стандартная ошибка среднего
  stderr = sem(sample)
  # Пороговое значение для нормального распределения
  h = stderr * norm.ppf((1 + confidence) / 2)


  return mmean, mmean - h, mmean + h


mmean, lower, upper = mean_confidence_interval(data)

print(f"Среднее: {mmean:.2f}, 95% доверительный интервал: [{lower:.2f}, {upper:.2f}]")

Среднее: 165.02, 95% доверительный интервал: [164.82, 165.21]


In [60]:
confidence_lvl = 0.80

mean, lower, upper = mean_confidence_interval(
    sample = data,
    confidence = confidence_lvl
)

print(f"Среднее: {mean:.2f}, {confidence_lvl * 100:.1f} % доверительный интервал: [{lower:.2f}, {upper:.2f}]")

Среднее: 165.02, 80.0 % доверительный интервал: [164.89, 165.15]


In [61]:
# Запусти код для визуальной проверки
std_dev = np.std(data)

fig = go.Figure()
# Гистограмма по нашим наблюдениям
fig.add_trace(
  go.Histogram(
      x=data,
      histnorm='probability density',
      name='Выборка роста студентов',
      opacity=0.6,
      marker_color='blue'
  )
)

# Построим линию плотности нормального распределения
x = np.linspace(mean - 4 * std_dev, mean + 4 * std_dev, 1000)
y = norm.pdf(x, mean, std_dev)
fig.add_trace(
  go.Scatter(
      x=x,
      y=y,
      mode='lines',
      name='Линия нормального распределения',
      line=dict(color='black')
  )
)


# Построение линии доверительного интервала
fig.add_shape(
  type="line",
  x0=lower,
  y0=0,
  x1=lower,
  y1=norm.pdf(lower, mean, std_dev),
  line=dict(color="red", width=2, dash="dash"),
  name="Левая граница доверительного интервала"
)

fig.add_shape(
  type="line",
  x0=upper,
  y0=0,
  x1=upper,
  y1=norm.pdf(upper, mean, std_dev),
  line=dict(color="red", width=2, dash="dash"),
  name="Левая граница доверительного интервала"
)

# Подписи значений границ доверительного интервала
fig.add_annotation(
  x=lower,
  y=norm.pdf(lower, mean, std_dev) + 0.005,
  text=f"{lower:.2f}",
  showarrow=False,
  font=dict(color="red")
)

fig.add_annotation(
  x=upper,
  y=norm.pdf(upper, mean, std_dev) + 0.005,
  text=f"{upper:.2f}",
  showarrow=False,
  font=dict(color="red")
)




fig.update_layout(
  title=f"Распределение роста студентов с доверительным интервалом уровня {confidence_lvl * 100:.1f} %",
  xaxis_title="Рост студента",
  yaxis_title="Плотность",
  showlegend=True,
)


fig.show()

Попробуйте посмотреть как меняются границы доверительного интервала от уровня доверия (confidence_level)